In [2]:
import pandas as pd
from tqdm import tqdm
import spacy
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor

In [3]:
tqdm.pandas()

In [4]:
train_df = pd.read_csv("data/train.tsv", sep='\t')
# test_df = pd.read_csv("data/test.tsv", sep='\t')

In [5]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-03-25 18:13:40.423485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-25 18:13:40.423707: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
You should consider upgrading via the 'C:\Users\Tyler\AppData\Local\Programs\Python\Python310\python3.exe -m pip install --upgrade pip' command.


In [6]:
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('stopwords')
stops = stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
train_set = train_df[:10000].copy()
testing_thing = train_df.iloc[20000:20050].copy()

In [8]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

In [9]:
knn = KNeighborsRegressor(n_neighbors=10)

In [10]:
def normalize(comment, lowercase, remove_stopwords):
    if comment == "No description yet":
        return ''
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops) and not word.is_punct:
                lemmatized.append(lemma)
    normalized = " ".join(lemmatized)
    return normalized

In [11]:
train_set['item_description_new'] = train_set['item_description'].progress_apply(normalize, lowercase=True, remove_stopwords=True)

100%|██████████| 10000/10000 [00:55<00:00, 180.42it/s]


In [12]:
tfidf_wm = tfidfvectorizer.fit_transform(train_set['item_description_new'].values.astype('U')) # https://stackoverflow.com/a/39308809/3675086

In [13]:
tfidf_tokens = tfidfvectorizer.get_feature_names_out()

In [14]:
df_tfidfvect = pd.DataFrame(
    data = tfidf_wm.toarray(),
    # index = ['Doc1','Doc2'],
    columns = tfidf_tokens
)

In [15]:
knn.fit(df_tfidfvect, train_set['price'])

KNeighborsRegressor(n_neighbors=10)

In [16]:
testing_thing['item_description_new'] = testing_thing['item_description'].progress_apply(normalize, lowercase=True, remove_stopwords=True)

100%|██████████| 50/50 [00:00<00:00, 181.65it/s]


In [17]:
tfidf_wm_2 = tfidfvectorizer.transform(testing_thing['item_description_new'].values.astype('U')) # https://stackoverflow.com/a/39308809/3675086

In [18]:
tfidf_tokens_2 = tfidfvectorizer.get_feature_names_out()

In [19]:
df_tfidfvect_2 = pd.DataFrame(
    data = tfidf_wm_2.toarray(),
    # index = ['Doc1','Doc2'],
    columns = tfidf_tokens
)

In [20]:
predictions = knn.predict(df_tfidfvect_2)

In [21]:
testing_thing['new_price'] = predictions

In [22]:
testing_thing

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,item_description_new,new_price
20000,20000,Blue crystal Heisenberg Funko pop!,3,Vintage & Collectibles/Toy/Action Figure,Funko,43.0,1,Minor flaws check pic for condition Bundle to ...,minor flaw check pic condition bundle save,54.9
20001,20001,Funko Pop STAR WARS EMPEROR PALPATINE NM,2,Kids/Toys/Action Figures & Statues,Funko,33.0,1,Near mint 9/10,near mint 9/10,23.7
20002,20002,Black Otter Box iPhone 6 / 6s,3,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,14.0,0,Excellent condition black Otter Box phone case...,excellent condition black otter box phone case...,16.0
20003,20003,32ft Red lights,1,Home/Home Décor/Lamps& Accessories,NaN,7.0,1,No description yet,,18.3
20004,20004,Dress,3,"Women/Dresses/Above Knee, Mini",FOREVER 21,22.0,0,Short dress with Aztec design on the front and...,short dress aztec design front back good condi...,22.8
20005,20005,Bundle do not buy,3,Kids/Girls 0-24 Mos/One-Pieces,Disney,39.0,0,Girls sleeping beauty set 9 months,girl sleep beauty set 9 month,18.3
20006,20006,UGG wedge,3,Women/Shoes/Sandals,UGG Australia,31.0,0,No description yet,,18.3
20007,20007,ON HOLD FOR ANNA!,1,Kids/Girls 0-24 Mos/One-Pieces,NaN,26.0,0,Purchasing all as one bundle 3 zip up sleepers...,purchase one bundle 3 zip sleeper new tag stai...,15.0
20008,20008,My Little Pony Castle,1,Kids/Toys/Dolls & Accessories,My Little Pony,25.0,1,Thank You for taking the time to look at my li...,thank take time look listing -brand new -low b...,22.7
20009,20009,Floral rug *New*,1,Home/Home Décor/Area Rugs & Pads,NaN,35.0,0,5x7 Bright/happy floral colored rug never been...,5x7 bright happy floral color rug never use mo...,13.9
